# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [29]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [31]:
raw_df = pd.read_csv('../output_data/WeatherPy_output/raw_weather.csv')
print(len(raw_df))
raw_df.head()

529


,City Name,City Latitude,City Longitude,Max Tempurature,Humidity,Cloud coverage,Wind speed,City Country,City datetime
0,Atuona,-9.80,-139.03,77.558,78.0,100.0,8.44,PF,1.609779e+09
1,Arlit,18.74,7.39,84.938,14.0,2.0,2.76,NE,1.609779e+09
2,Clyde River,70.47,-68.59,-18.400,82.0,75.0,7.20,CA,1.609779e+09
3,Kapaa,22.08,-159.32,73.400,78.0,20.0,4.10,US,1.609779e+09
4,Lompoc,34.64,-120.46,46.400,100.0,90.0,2.60,US,1.609779e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [32]:
import gmaps
gmaps.configure(api_key=g_key)

In [34]:
coords = raw_df[['City Latitude', 'City Longitude']].fillna(0)
humidity = raw_df['Humidity']

fig = gmaps.figure(map_type='HYBRID')

heatmap_layer = gmaps.heatmap_layer(coords, weights=humidity, dissipating=False, max_intensity=10, point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [35]:
ideal_temp_mask = (raw_df['Max Tempurature'] < 80) & (raw_df['Max Tempurature'] > 70)
ideal_wind_mask = raw_df['Wind speed'] < 10
ideal_cloud_mask = raw_df['Cloud coverage'] == 0 

ideal_spots_df = raw_df[ideal_temp_mask & ideal_wind_mask & ideal_cloud_mask]
ideal_spots_df.index = ideal_spots_df['City Name']
ideal_spots_df = ideal_spots_df.rename(columns={'City Name': 'City'})
ideal_spots_df.head()

,City,City Latitude,City Longitude,Max Tempurature,Humidity,Cloud coverage,Wind speed,City Country,City datetime
City Name,,,,,,,,,
Pimentel,Pimentel,-6.84,-79.93,78.800,61.0,0.0,2.10,PE,1.609779e+09
Umm Kaddadah,Umm Kaddadah,13.60,26.69,77.612,19.0,0.0,4.96,SD,1.609779e+09
Nouadhibou,Nouadhibou,20.93,-17.03,75.200,36.0,0.0,3.60,MR,1.609779e+09
Cidreira,Cidreira,-30.18,-50.21,77.810,66.0,0.0,7.96,BR,1.609779e+09
La Libertad,La Libertad,-2.23,-80.90,78.800,74.0,0.0,2.60,EC,1.609779e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
ideal_coords = ideal_spots_df[['City Latitude','City Longitude']]
city_names = ideal_spots_df['City']
string_coords = []
responses = []

for vacation_spot in range(len(ideal_coords)):
    str_coord = f'{str(ideal_coords.iloc[vacation_spot, 0])}, {str(ideal_coords.iloc[vacation_spot,1])}'
    string_coords.append(str_coord)

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_search = "Hotel"
target_radius = 5000
target_type = "hotel"

for city in city_names:
        print(f'Searching for hotels within {target_radius}m of {city} with Google Places API')
        
for vacation_spot in string_coords: 
    
    params = {
        "location": vacation_spot,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    response = requests.get(base_url, params=params)
    responses.append(response)

Searching for hotels within 5000m of Pimentel with Google Places API
Searching for hotels within 5000m of Umm Kaddadah with Google Places API
Searching for hotels within 5000m of Nouadhibou with Google Places API
Searching for hotels within 5000m of Cidreira with Google Places API
Searching for hotels within 5000m of La Libertad with Google Places API
Searching for hotels within 5000m of Port Elizabeth with Google Places API
Searching for hotels within 5000m of Manakara with Google Places API
Searching for hotels within 5000m of La Constancia with Google Places API
Searching for hotels within 5000m of Yulara with Google Places API
Searching for hotels within 5000m of Bilma with Google Places API


In [37]:
clean_responses = [resp for resp in responses if resp.json()['status'] != 'ZERO_RESULTS']

In [38]:
from citipy import citipy

# Getting coords of the FIRST HOTEL OF EACH SEARCH WITHIN 5000m OF CITY 
hotel_info = {'Hotel Name': [],
              'Lat': [],
              'Lng': [],
              'City': []}
    
for result_list in clean_responses:
    hotels = result_list.json()
    first_hotel = hotels['results'][0]
    
    hotel_info['Hotel Name'].append(first_hotel['name'])
    hotel_info['Lat'].append(first_hotel['geometry']['location']['lat'])
    hotel_info['Lng'].append(first_hotel['geometry']['location']['lng'])

# Getting cities of hotels (hotel may not be within the city of search)
for coord_pair in zip(hotel_info['Lat'], hotel_info['Lng']):
    city = citipy.nearest_city(coord_pair[0], coord_pair[1]).city_name
    hotel_info['City'].append(city.title())

In [40]:
# Make hotel_df w/ hotel_name, lat, lng, country, city_name cols
countries = ideal_spots_df[['City','City Country']]

hotel_df = pd.DataFrame(hotel_info)
hotel_df = pd.merge(hotel_df, countries, on='City',how='inner')
hotel_df = hotel_df.rename(columns={'City Country': 'Country'})

try:
    hotel_df = hotel_df.sample(10)
except:
    hotel_df = hotel_df

len(hotel_df)

7

In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [28]:
# Add marker layer ontop of heat map

hotel_names = hotel_df['Hotel Name'].values.tolist()

marker_layer = gmaps.marker_layer(locations, hotel_names, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))